In [1]:
import os
from matplotlib import patches
import numpy as np
from numpy.core.fromnumeric import _cumprod_dispatcher
import pandas as pd
import matplotlib.pyplot as plt
import cv2
from PIL import Image, ImageOps

import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout

In [2]:
### Metadata for labels ##################################################################################################
# Classes: 'Car', 'Van', 'Truck', 'Pedestrian', 'Person_sitting', 'Cyclist', 'Tram', 'Misc' or 'DontCare'
# Total classes: 7
# Label data (by index):
#   0           string - class/label name                               (required)
#   1           float - describing if the object is in frame
#   2           integer - describes object occlusion from 0 to 3
#   3           float - observation angle
#   4,5,6,7     float - 2d bounding box in pixel coordinates            (required)
#   8,9,10      float - object dimensions in 3D
#   11,12,13    float - object location in 3D
#   14          float - rotation of object around y axis
#   15          float - score (unused)

# Training Image Location: /data_object_image_2/training/image_2
# Testing Image Location:/data_object_image_2/testing/image_2
# Label Location: /data_object_label_2/training/label_2

# Image name format: "000000.png"
# Label name format: "000000.txt"


### TO DO #####################################
# >The class labels need to be int values (0 - 6). Currently they are strings.
# >The final Y array needs to only be int class values with an associated image
#   -May split current Y array into two. One for class, one with class and meta data
# >Image resizing may not be the best rn. Test cases and see options for high aspect ratios


### Defining variables and loading training data #########################################################################
# print(tf.test.gpu_device_name())  # For testing GPU accessibility on linux

In [4]:
# Directory setup
cur_dir = os.getcwd()
train_img_dir = "data_object_image_2/training/image_2"
test_img_dir = "data_object_image_2/testing/image_2"
label_dir = "data_object_label_2/training/label_2"

In [78]:
# Gets the paths for the individual images in the training data
train_img_path = os.path.join(cur_dir, train_img_dir)
images = os.listdir(train_img_path)[:10]

In [79]:
# Gets the paths for the training labels
label_path = os.path.join(cur_dir, label_dir)
labels = os.listdir(label_path)[:10]

In [80]:
# Saves the label class and bounding box coords from each file
X = []
Y = []

for data in labels:
    try:
        lines = []
        with open(os.path.join(label_path,data)) as f:
            lines = f.readlines()

        for line in lines:
            inst = line.split()
            if inst[0] != "DontCare" and inst[0] != "Misc": # further limitations given the dataset such as no occulsions may be added
                # [dataframe, class, top, left, bottom right] for each index
                # Y needs to be changed to only have the class ints
                Y.append([(str(data).rsplit('.',1)[0]), inst[0], round(float(inst[4])), round(float(inst[5])), round(float(inst[6])), round(float(inst[7]))])
    
    except Exception as e:
        print(e)


for i in range(len(Y)):
    try: 
        inst = Y[i]
        img_file = inst[0] + '.png'
        directory = os.path.join(train_img_path, img_file)

        img = Image.open(directory)
        img_res = img.crop((inst[2], inst[3], inst[4], inst[5]))
        shape = (30, 30)
        img_res = img_res.resize(shape)
        img_res = ImageOps.grayscale(img_res)
        img_res = np.array(img_res)
        X.append(img_res)

    except Exception as e:
        print(e)
        
    

In [81]:
# img_res.show()

# print(len(X))
# print(len(Y))
X = np.array(X)
print(X.shape)

Y = np.array(Y)
print(Y.shape)

(26, 30, 30)
(26, 6)


In [117]:
categories_dict = {}
categories = []
num_categories = 0

for label in Y:
    if label[1] in categories_dict:
        categories.append(categories_dict[label[1]])
    else:
        categories_dict[label[1]] = num_categories
        num_categories += 1
        categories.append(categories_dict[label[1]])
Y = catagories

In [119]:
print(X.shape)
print(len(Y))

(26, 30, 30)
26


In [129]:
### Creating and training the model ###################################################################################
# Sets aside 20% of the original data to use for testing, all else is used to train. Randomizes based on seed (42 here).
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

y_train = to_categorical(y_train, 7)
print(y_train.shape)
y_test = to_categorical(y_test, 7)
print(y_test.shape)

opt = keras.optimizers.Adam(lr = 0.0008)  # The learning rate details for the network

model = Sequential()
model.add(Conv2D(filters=16, kernel_size=(5,5), activation='relu', input_shape=(26,30,30)))  # input shape MUST be size of all input imgs
model.add(Conv2D(filters=16, kernel_size=(5,5), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(rate=0.25))
model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu'))
model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(rate=0.25))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(rate=0.5))
model.add(Dense(7, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

epochs = 10   # How many iterations the model will run
history = model.fit(x_train, y_train, batch_size=64, epochs=epochs, validation_data=(x_test, y_test)) # Saves metadata for later
# model.save('kitti_cnn.h5')


(20, 7)
(6, 7)
Epoch 1/10


ValueError: in user code:

    /home/nick/anaconda3/envs/tf-gpu/lib/python3.9/site-packages/tensorflow/python/keras/engine/training.py:805 train_function  *
        return step_function(self, iterator)
    /home/nick/anaconda3/envs/tf-gpu/lib/python3.9/site-packages/tensorflow/python/keras/engine/training.py:795 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /home/nick/anaconda3/envs/tf-gpu/lib/python3.9/site-packages/tensorflow/python/distribute/distribute_lib.py:1259 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /home/nick/anaconda3/envs/tf-gpu/lib/python3.9/site-packages/tensorflow/python/distribute/distribute_lib.py:2730 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /home/nick/anaconda3/envs/tf-gpu/lib/python3.9/site-packages/tensorflow/python/distribute/distribute_lib.py:3417 _call_for_each_replica
        return fn(*args, **kwargs)
    /home/nick/ana